<a href="https://colab.research.google.com/github/nickprock/appunti_data_science/blob/master/spacy/SpaCy_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SpaCy Wrapper per HuggingFace

<br>

![spacyHF](https://huggingface.co/blog/assets/23_spacy/thumbnail.png)

<br>

In questo notebook c'è spiegato in poche righe come prendere un transformer da Huggingface Hub, wrapparlo in Spacy, condividerlo sull'Hub e utilizzarlo.

## Downloads

Per prima cosa scarichiamo le librerie:
* spacy-huggingface-hub
* spacy-wrap

> spacy-wrap funziona solo con python>=3.8 quindi bisogna fare un check.

In [1]:
!python --version

Python 3.10.11


In [2]:
!pip install git+https://github.com/KennethEnevoldsen/spacy-wrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/KennethEnevoldsen/spacy-wrap to /tmp/pip-req-build-5wn0lmm3
  Running command git clone --filter=blob:none --quiet https://github.com/KennethEnevoldsen/spacy-wrap /tmp/pip-req-build-5wn0lmm3
  Resolved https://github.com/KennethEnevoldsen/spacy-wrap to commit db00ffdeffbef7d0c7e041d1bdcd966f97a9dac1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.7/190.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
  Created wheel for spacy-wrap: filen

In [3]:
!pip install spacy-huggingface-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download del modello dall'Hub

Per questo test userò un modello per la NER in italiano di cui avevo fatto fine-tuning.

* Per prima cosa importiamo le librerie e inizializziamo una pipeline `blank` per l'italiano.

* Secondo step: creiamo il nostro config dove specifichiamo il modello che vogliamo utilizzare e il checkpoint sull'hub.

* Ultimo step: aggiungiamo il nostro *config* alla pipeline con `nlp.add_pipe`

Ora possiamo già usare il modello.

> N.B. il `config` può essere molto più complesso, consiglio la [documentazione ufficiale su spacy wrap](https://spacy.io/universe/project/spacy-wrap)

In [4]:
import spacy
import spacy_wrap
nlp = spacy.blank("it")

# specify model from the hub
config = {"model": {"name": "nickprock/bert-italian-finetuned-ner"}}

# add it to the pipe
nlp.add_pipe("token_classification_transformer", config=config)

doc = nlp("Ciao, sono Giacomo. Vivo a Milano e lavoro da Armani.")

print(doc.ents)

(Giacomo, Milano, Armani)


In [5]:
nlp.pipe_names

['token_classification_transformer']

## Condividi il modello sull'Hub

Se volessi non utilizzare ogni volta SpaCy Wrap e anzi volessi rendere disponibile il mio lavoro alla community posso condividere semplicemente il mio modello su HuggingFace Hub.

Per prima cosa salvo il modello sul disco.

> N.B. tutto il notebook è creato e usato su colab

In [7]:
config = nlp.config
bytes_data = nlp.to_bytes()

In [8]:
nlp.to_disk("/content/sample_data/it_spacy_ner_trf")

> Ho trovato un piccolo bug (noto alla community) che impediva di fare la build dei modelli correttamente, nella prossima cella la soluzione per aggirarlo.

In [9]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

per fare la build si può usare la prossima cella, ricordatevi di inserire il parametro `--name` altrimenti SpaCy darà il nome di default, in questo caso `it_pipeline-0.0.0`.

In [10]:
!python -m spacy package /content/sample_data/it_spacy_ner_trf /content/sample_data/ --name 'spacy_ner_trf' --build wheel 

2023-05-19 08:50:29.554827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Building package artifacts: wheel
✔ Including 2 package requirement(s) from meta and config
spacy-transformers>=1.2.3,<1.3.0, spacy-wrap>=1.4.2,<1.5.0
✔ Loaded meta.json from file
/content/sample_data/it_spacy_ner_trf/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'it_spacy_ner_trf-0.0.0'
/content/sample_data/it_spacy_ner_trf-0.0.0
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/it_spacy_ner_trf
copying it_spacy_ner_trf/__init__.py -> build/lib/it_spacy_ner_trf
creating build/lib/it_spacy_ner_trf/it_spacy_ner_trf-0.0.0
copying it_spacy_ner_trf/it_spacy_ner_trf-0.0.0/tokenizer -> build/lib/it_spacy_ner_trf/it_spacy_ner_trf-0.0.0
copying it_spacy_ner_trf/it_spacy_ner_trf-0.0.0/README.md -> build/lib/it_spacy_ner_trf/it_spacy_ner_trf-0.0.0
copying it_spacy_ner_trf/it_sp

### Condividere tramite Spacy HuggingFace Hub

Dopo essersi autentucati col proprio token HuggingFace basa importare dalla libreria la funzione `push` e applicarla al file `.whl` che abbiamo creato con la build.

Ci facciamo stampare l'URL del checkpoint sull'hub per vedere se il processo è andato a buon fine.

In [12]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 45, in main
    service.run()
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/c

In [ ]:
from spacy_huggingface_hub import push

result = push("/content/sample_data/it_spacy_ner_trf-0.0.0/dist/it_spacy_ner_trf-0.0.0-py3-none-any.whl")
print(result["url"])

## Scarica dall'Hub e utilizza direttamente in SpaCy

Tutto pronto ora possiamo utilizzare il nostro transformer in SpaCy come abbiamo sempre fatto con le pipeline di questa libreria.

In [11]:
!pip install https://huggingface.co/nickprock/it_spacy_ner_trf/resolve/main/it_spacy_ner_trf-any-py3-none-any.whl

# Using spacy.load().
import spacy
nlp = spacy.load("it_spacy_ner_trf")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.2/406.2 MB 2.2 MB/s eta 0:00:00


In [21]:
doc = nlp("Domenica andrò a Firenze con Giovanna a guardare la Fiorentina.")

print(doc.ents)

(Firenze, Giovanna, Fiorentina)


In [22]:
for ent in doc.ents:
    print(ent.text, ent.label_)

Firenze LOC
Giovanna PER
Fiorentina LOC
